## Real Reviews
### Optimizing Business Reviews Using NLP
### (In Progress) - This is currently a scratchpad

In [1]:
import json
import pandas as pd
import nltk
import sklearn.metrics
import pickle
import psycopg2
from yelpapi import YelpAPI
from pprint import pprint
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Read the Yelp reviews dataset into a DataFrame
# Using 100,000 records to build model
with open('yelp_academic_dataset_review.json') as f:    
    d = [json.loads(f.readline().strip()) for x in range(100000)]

In [3]:
# Create the DataFrame
df = pd.DataFrame(d)

## Look Up Business Data Using Yelp API

In [4]:
# How many unique Businesses?
len(df['business_id'].unique())

55117

In [19]:
# Initialize the Yelp API
yelp_id = ""
yelp_secret = ""

with open('pvt.csv') as f:
    for line in f:
        l = line.strip().split(',')
        if l[0] == "yelp_id":
            yelp_id = l[1]
        if l[0] == "yelp_secret":
            yelp_secret = l[1]

yelp_api = YelpAPI(yelp_id, yelp_secret)

In [49]:
# Choose a business and store the response from API
bid = df['business_id'].sample(1).iloc[0]
response = yelp_api.business_query(id=bid)
response

{u'categories': [{u'alias': u'foodtrucks', u'title': u'Food Trucks'},
  {u'alias': u'donuts', u'title': u'Donuts'},
  {u'alias': u'bakeries', u'title': u'Bakeries'}],
 u'coordinates': {u'latitude': 40.1071641393929,
  u'longitude': -88.2230068528675},
 u'display_phone': u'(630) 946-4306',
 u'id': u'pandamonium-doughnuts-urbana',
 u'image_url': u'https://s3-media1.fl.yelpcdn.com/bphoto/s_P15JNk6Q3xVMhlXx-c7Q/o.jpg',
 u'is_claimed': False,
 u'is_closed': False,
 u'location': {u'address1': u'',
  u'address2': u'',
  u'address3': u'',
  u'city': u'Urbana',
  u'country': u'US',
  u'cross_streets': u'',
  u'display_address': [u'Urbana, IL 61801'],
  u'state': u'IL',
  u'zip_code': u'61801'},
 u'name': u'Pandamonium Doughnuts',
 u'phone': u'+16309464306',
 u'photos': [u'https://s3-media3.fl.yelpcdn.com/bphoto/s_P15JNk6Q3xVMhlXx-c7Q/o.jpg',
  u'https://s3-media4.fl.yelpcdn.com/bphoto/TnXIFT0mK3HCPieZIZn7kg/o.jpg',
  u'https://s3-media4.fl.yelpcdn.com/bphoto/suCiXbd8r4tBBtHPkZ5FIQ/o.jpg'],
 u'p

In [39]:
# Variables from 'response' that we will need
city = response['location']['city']
country = response['location']['country']
name = response['name']

In [40]:
print(city)
print(country)
print(name)

Phoenix
US
Palatte


## Sentiment Analysis

In [47]:
# Let's just look at our sample business for an example
df_bid = df[df['business_id']==bid]
df_bid

business_id  cool        date  funny  \
86971  EhK37wNBtzk64S8kksL9Og     2  2008-11-23      2   
86972  EhK37wNBtzk64S8kksL9Og     2  2008-01-29      0   
86973  EhK37wNBtzk64S8kksL9Og     0  2008-12-02      0   
86974  EhK37wNBtzk64S8kksL9Og     1  2008-05-07      1   
86975  EhK37wNBtzk64S8kksL9Og     1  2008-04-18      0   
86976  EhK37wNBtzk64S8kksL9Og     3  2008-05-29      0   
86977  EhK37wNBtzk64S8kksL9Og     2  2008-09-28      2   
86978  EhK37wNBtzk64S8kksL9Og     1  2009-02-08      0   
86979  EhK37wNBtzk64S8kksL9Og     1  2008-11-23      1   
86980  EhK37wNBtzk64S8kksL9Og    10  2008-03-21      5   
86981  EhK37wNBtzk64S8kksL9Og     3  2008-05-29      5   
86982  EhK37wNBtzk64S8kksL9Og     1  2007-10-26      0   
86983  EhK37wNBtzk64S8kksL9Og    12  2008-09-09      4   
86984  EhK37wNBtzk64S8kksL9Og     2  2008-06-17      1   
86985  EhK37wNBtzk64S8kksL9Og     2  2008-08-25      0   
86986  EhK37wNBtzk64S8kksL9Og     5  2008-08-21      2   
86987  EhK37wNBtzk64S8kksL9Og     1  2008-10-09      1   
86988  EhK37wNBtzk64S8kksL9Og     2  2008-09-15      2   
86989  EhK37wNBtzk64S8kksL9Og     0  2009-03-10      0   
86990  EhK37wNBtzk64S8kksL9Og     3  2008-06-26      2   
86991  EhK37wNBtzk64S8kksL9Og     1  2008-12-06      0   
86992  EhK37wNBtzk64S8kksL9Og     8  2007-12-15      5   
86993  EhK37wNBtzk64S8kksL9Og     2  2008-10-12      1   
86994  EhK37wNBtzk64S8kksL9Og     2  2008-08-26      1   
86995  EhK37wNBtzk64S8kksL9Og     3  2007-10-24      0   
86996  EhK37wNBtzk64S8kksL9Og     2  2008-09-13      3   
86997  EhK37wNBtzk64S8kksL9Og     2  2008-04-04      5   
86998  EhK37wNBtzk64S8kksL9Og     2  2009-01-28      0   
86999  EhK37wNBtzk64S8kksL9Og     2  2008-09-26      2   
87000  EhK37wNBtzk64S8kksL9Og     1  2008-10-08      0   
...                       ...   ...         ...    ...   
87003  EhK37wNBtzk64S8kksL9Og     3  2007-11-23      0   
87004  EhK37wNBtzk64S8kksL9Og     3  2008-12-26      3   
87005  EhK37wNBtzk64S8kksL9Og     1  2008-09-18      1   
87006  EhK37wNBtzk64S8kksL9Og     4  2008-11-20      4   
87007  EhK37wNBtzk64S8kksL9Og     6  2008-01-26      3   
87008  EhK37wNBtzk64S8kksL9Og     2  2008-04-11      1   
87009  EhK37wNBtzk64S8kksL9Og     8  2007-09-09      2   
87010  EhK37wNBtzk64S8kksL9Og     3  2008-10-29      3   
87011  EhK37wNBtzk64S8kksL9Og     0  2008-09-17      0   
87012  EhK37wNBtzk64S8kksL9Og     4  2007-11-10      1   
87013  EhK37wNBtzk64S8kksL9Og     3  2008-09-16      0   
87014  EhK37wNBtzk64S8kksL9Og     1  2008-09-06      0   
87015  EhK37wNBtzk64S8kksL9Og     1  2009-03-24      0   
87016  EhK37wNBtzk64S8kksL9Og     0  2008-09-21      0   
87017  EhK37wNBtzk64S8kksL9Og     2  2008-05-09      0   
87018  EhK37wNBtzk64S8kksL9Og     1  2008-07-21      0   
87019  EhK37wNBtzk64S8kksL9Og     3  2008-07-06      3   
87020  EhK37wNBtzk64S8kksL9Og     8  2007-10-14      4   
87021  EhK37wNBtzk64S8kksL9Og     2  2007-10-13      0   
87022  EhK37wNBtzk64S8kksL9Og     1  2009-02-23      1   
87023  EhK37wNBtzk64S8kksL9Og     3  2008-08-24      1   
87024  EhK37wNBtzk64S8kksL9Og     3  2008-12-06      3   
87025  EhK37wNBtzk64S8kksL9Og     0  2009-02-17      0   
87026  EhK37wNBtzk64S8kksL9Og     0  2008-12-17      0   
87027  EhK37wNBtzk64S8kksL9Og     1  2007-11-27      0   
87028  EhK37wNBtzk64S8kksL9Og     2  2008-03-09      1   
87029  EhK37wNBtzk64S8kksL9Og     2  2008-08-06      0   
87030  EhK37wNBtzk64S8kksL9Og     1  2008-02-01      0   
87031  EhK37wNBtzk64S8kksL9Og     3  2007-07-20      0   
87032  EhK37wNBtzk64S8kksL9Og     3  2009-02-10      3   

                    review_id  stars  \
86971  ddZV0X0c4n3ejNtGub5x-A      5   
86972  kVqUau8UPavliH4KvEGpSw      4   
86973  MMbul_52OwAnjE7Kh95eQg      3   
86974  4SUwIsCiG_Pj6LzlT16tKw      3   
86975  OCWAt4XGiLRB1BB4A3fMnA      5   
86976  y9mR4MPmlOStYWAhngOG-Q      5   
86977  rvRpBfo5RwbLD8yjgIH7WQ      2   
86978  WqoQ-dUbSK0YOsv3uYtezQ      5   
86979  H3fZ3yu9JYnjor_TgCGjEw      1   
86980  yZ2cQTXYVDjEAs

In [48]:
# How many records in this database?
len(df_bid)

62

In [43]:
# Add column with polarity ranking
# This represents how strong the opinion of the review is in either
#   direction (favorable or unfavorable)
df_bid['polarity'] = [TextBlob(i).sentiment[0] for i in df_bid['text']]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [44]:
# Add column with subjectivity ranking
# This represents how 'objective' the review is, or
#  whether the language indicates that it is very
#  subjective (opinions as opposed to factual)
df_bid['subjectivity'] = [TextBlob(i).sentiment[1] for i in df_bid['text']]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [45]:
# View the new DataFrame
df_bid

business_id  cool        date  funny  \
86971  EhK37wNBtzk64S8kksL9Og     2  2008-11-23      2   
86972  EhK37wNBtzk64S8kksL9Og     2  2008-01-29      0   
86973  EhK37wNBtzk64S8kksL9Og     0  2008-12-02      0   
86974  EhK37wNBtzk64S8kksL9Og     1  2008-05-07      1   
86975  EhK37wNBtzk64S8kksL9Og     1  2008-04-18      0   
86976  EhK37wNBtzk64S8kksL9Og     3  2008-05-29      0   
86977  EhK37wNBtzk64S8kksL9Og     2  2008-09-28      2   
86978  EhK37wNBtzk64S8kksL9Og     1  2009-02-08      0   
86979  EhK37wNBtzk64S8kksL9Og     1  2008-11-23      1   
86980  EhK37wNBtzk64S8kksL9Og    10  2008-03-21      5   
86981  EhK37wNBtzk64S8kksL9Og     3  2008-05-29      5   
86982  EhK37wNBtzk64S8kksL9Og     1  2007-10-26      0   
86983  EhK37wNBtzk64S8kksL9Og    12  2008-09-09      4   
86984  EhK37wNBtzk64S8kksL9Og     2  2008-06-17      1   
86985  EhK37wNBtzk64S8kksL9Og     2  2008-08-25      0   
86986  EhK37wNBtzk64S8kksL9Og     5  2008-08-21      2   
86987  EhK37wNBtzk64S8kksL9Og     1  2008-10-09      1   
86988  EhK37wNBtzk64S8kksL9Og     2  2008-09-15      2   
86989  EhK37wNBtzk64S8kksL9Og     0  2009-03-10      0   
86990  EhK37wNBtzk64S8kksL9Og     3  2008-06-26      2   
86991  EhK37wNBtzk64S8kksL9Og     1  2008-12-06      0   
86992  EhK37wNBtzk64S8kksL9Og     8  2007-12-15      5   
86993  EhK37wNBtzk64S8kksL9Og     2  2008-10-12      1   
86994  EhK37wNBtzk64S8kksL9Og     2  2008-08-26      1   
86995  EhK37wNBtzk64S8kksL9Og     3  2007-10-24      0   
86996  EhK37wNBtzk64S8kksL9Og     2  2008-09-13      3   
86997  EhK37wNBtzk64S8kksL9Og     2  2008-04-04      5   
86998  EhK37wNBtzk64S8kksL9Og     2  2009-01-28      0   
86999  EhK37wNBtzk64S8kksL9Og     2  2008-09-26      2   
87000  EhK37wNBtzk64S8kksL9Og     1  2008-10-08      0   
...                       ...   ...         ...    ...   
87003  EhK37wNBtzk64S8kksL9Og     3  2007-11-23      0   
87004  EhK37wNBtzk64S8kksL9Og     3  2008-12-26      3   
87005  EhK37wNBtzk64S8kksL9Og     1  2008-09-18      1   
87006  EhK37wNBtzk64S8kksL9Og     4  2008-11-20      4   
87007  EhK37wNBtzk64S8kksL9Og     6  2008-01-26      3   
87008  EhK37wNBtzk64S8kksL9Og     2  2008-04-11      1   
87009  EhK37wNBtzk64S8kksL9Og     8  2007-09-09      2   
87010  EhK37wNBtzk64S8kksL9Og     3  2008-10-29      3   
87011  EhK37wNBtzk64S8kksL9Og     0  2008-09-17      0   
87012  EhK37wNBtzk64S8kksL9Og     4  2007-11-10      1   
87013  EhK37wNBtzk64S8kksL9Og     3  2008-09-16      0   
87014  EhK37wNBtzk64S8kksL9Og     1  2008-09-06      0   
87015  EhK37wNBtzk64S8kksL9Og     1  2009-03-24      0   
87016  EhK37wNBtzk64S8kksL9Og     0  2008-09-21      0   
87017  EhK37wNBtzk64S8kksL9Og     2  2008-05-09      0   
87018  EhK37wNBtzk64S8kksL9Og     1  2008-07-21      0   
87019  EhK37wNBtzk64S8kksL9Og     3  2008-07-06      3   
87020  EhK37wNBtzk64S8kksL9Og     8  2007-10-14      4   
87021  EhK37wNBtzk64S8kksL9Og     2  2007-10-13      0   
87022  EhK37wNBtzk64S8kksL9Og     1  2009-02-23      1   
87023  EhK37wNBtzk64S8kksL9Og     3  2008-08-24      1   
87024  EhK37wNBtzk64S8kksL9Og     3  2008-12-06      3   
87025  EhK37wNBtzk64S8kksL9Og     0  2009-02-17      0   
87026  EhK37wNBtzk64S8kksL9Og     0  2008-12-17      0   
87027  EhK37wNBtzk64S8kksL9Og     1  2007-11-27      0   
87028  EhK37wNBtzk64S8kksL9Og     2  2008-03-09      1   
87029  EhK37wNBtzk64S8kksL9Og     2  2008-08-06      0   
87030  EhK37wNBtzk64S8kksL9Og     1  2008-02-01      0   
87031  EhK37wNBtzk64S8kksL9Og     3  2007-07-20      0   
87032  EhK37wNBtzk64S8kksL9Og     3  2009-02-10      3   

                    review_id  stars  \
86971  ddZV0X0c4n3ejNtGub5x-A      5   
86972  kVqUau8UPavliH4KvEGpSw      4   
86973  MMbul_52OwAnjE7Kh95eQg      3   
86974  4SUwIsCiG_Pj6LzlT16tKw      3   
86975  OCWAt4XGiLRB1BB4A3fMnA      5   
86976  y9mR4MPmlOStYWAhngOG-Q      5   
86977  rvRpBfo5RwbLD8yjgIH7WQ      2   
86978  WqoQ-dUbSK0YOsv3uYtezQ      5   
86979  H3fZ3yu9JYnjor_TgCGjEw      1   
86980  yZ2cQTXYVDjEAs

## NLP Model for Simplifying Reviews

In [4]:
# We will only look at 5 star or 1 start reviews for training the model
# This implies that all 5 star reviews are definitely "favorable",
#  while 1 star reviews are definitely "unfavorable"
df_bin = df[(df['stars'] == 5) | (df['stars'] == 1)][['text', 'stars']]

In [ ]:
# How many records in the new DataFrame?
len(df_bin)

In [5]:
# Binarize the star rankings (1 = Favorable, 0 = Unfavorable)
df_bin['stars'] = df_bin['stars'].map(lambda x: 1 if x == 5 else 0)

In [7]:
# Renumber the index
df_bin.index = range(len(df_bin))

In [135]:
# Create our Tfidf Vectorizer which will be used to
#  rank the importance of words for our classifier
vectorizer = TfidfVectorizer(ngram_range=(1,2), strip_accents='unicode')
vectorizer.fit(df_bin['text'])
x = vectorizer.transform(df_bin['text'])

NameError: name 'df_bin' is not defined

In [11]:
# What are the details of our sparse matrix?
x

<540612x5847148 sparse matrix of type '<type 'numpy.float64'>'
	with 89101432 stored elements in Compressed Sparse Row format>

In [161]:
# Split sample into Train and Test sets for cross validation
X_train, X_test, y_train, y_test = train_test_split(x, df_bin['stars'], test_size=0.3, stratify=df_bin['stars'])

In [81]:
%%timeit -n1 -r1
# Random Forests Cross Validaton on F1 Score
forest = RandomForestClassifier()
forest_acc = cross_val_score(forest, X_train, y_train, cv=1, scoring='f1')
print(forest_acc)

[ 0.91031227  0.90340087  0.90697251  0.908728    0.89891304  0.90793536
  0.90660841  0.90048587  0.90462428  0.90925689]
1 loop, best of 1: 25min 41s per loop


In [ ]:
# Train a Random Forests model on the entire dataset
forest = RandomForestClassifier()
forest.fit(x, df_bin['stars'])

In [127]:
%%timeit -n1 -r1
# Multinomial Naive Bayes Cross Validaton on F1 Score
nb = MultinomialNB()
nb_acc = cross_val_score(nb, X_train, y_train, cv=10, scoring='f1')
print(nb_acc)

[ 0.82130584  0.82130584  0.82068966  0.82210708  0.82210708  0.82210708
  0.82210708  0.82210708  0.82352941  0.82210708]
1 loop, best of 1: 611 ms per loop


In [50]:
# Pickle the Random Forests model
with open('model/forest.pkl', 'wb') as f:
    pickle.dump(forest, f, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'forest' is not defined

In [ ]:
# Pickle the Tfidf Vectorizer
with open('model/vectorizer.pkl', 'wb') as f:
    pickle.dump(forest, f, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
# Load the Random Forests Model
with open('model/forest.pkl', 'rb') as f:
    forest = pickle.load(f)

In [15]:
# Load the Tfidf Vectorizer
with open('model/vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [20]:
# Test the model
forestvec = vectorizer.transform(['good food great food bad service horrible people'])
forest.predict(forestvec)

array([0])